In [4]:
# ANN MODEL

import pandas as pd
import numpy as np
import joblib

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, Dropout, GlobalAveragePooling1D
from tensorflow.keras.callbacks import EarlyStopping


data_file = "suicide_detection_rare_word_removed.csv"
df = pd.read_csv(data_file)

print("Dataset loaded successfully")

df["text"] = df["text"].astype(str)

X = df["text"]
y = df["class"]

y = y.map({"non-suicide": 0, "suicide": 1})

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("Train/Test split completed")

MAX_WORDS = 20000
MAX_LEN = 200

tokenizer = Tokenizer(num_words=MAX_WORDS, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding="post")
X_test_pad = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding="post")

print("Tokenization & padding completed")


model = Sequential([
    Embedding(input_dim=MAX_WORDS, output_dim=128, input_length=MAX_LEN),
    GlobalAveragePooling1D(),
    Dense(128, activation="relu"),
    Dropout(0.5),
    Dense(1, activation="sigmoid")
])

model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

model.summary()


early_stop = EarlyStopping(
    monitor="val_loss",
    patience=2,
    restore_best_weights=True
)

history = model.fit(
    X_train_pad,
    y_train,
    epochs=10,
    batch_size=128,
    validation_split=0.1,
    callbacks=[early_stop],
    verbose=1
)


y_pred_prob = model.predict(X_test_pad)
y_pred = (y_pred_prob > 0.5).astype(int)

accuracy = accuracy_score(y_test, y_pred)
print("\nAccuracy:", accuracy)

print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=["non-suicide", "suicide"]))


model.save("ann_model.h5")
joblib.dump(tokenizer, "ann_tokenizer.pkl")

print("\nANN model and tokenizer saved successfully")



Dataset loaded successfully
Train/Test split completed
Tokenization & padding completed


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_1      │ ?                      │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
1305/1305 ━━━━━━━━━━━━━━━━━━━━ 35s 26ms/step - accuracy: 0.8259 - loss: 0.3817 - val_accuracy: 0.9007 - val_loss: 0.2517
Epoch 2/10
1305/1305 ━━━━━━━━━━━━━━━━━━━━ 41s 26ms/step - accuracy: 0.9169 - loss: 0.2212 - val_accuracy: 0.9328 - val_loss: 0.1782
Epoch 3/10
1305/1305 ━━━━━━━━━━━━━━━━━━━━ 34s 26ms/step - accuracy: 0.9272 - loss: 0.1919 - val_accuracy: 0.9354 - val_loss: 0.1707
Epoch 4/10
1305/1305 ━━━━━━━━━━━━━━━━━━━━ 36s 28ms/step - accuracy: 0.9342 - loss: 0.1762 - val_accuracy: 0.9350 - val_loss: 0.1675
Epoch 5/10
1305/1305 ━━━━━━━━━━━━━━━━━━━━ 40s 26ms/step - accuracy: 0.9318 - loss: 0.1758 - val_accuracy: 0.9300 - val_loss: 0.1871
Epoch 6/10
1305/1305 ━━━━━━━━━━━━━━━━━━━━ 36s 27ms/step - accuracy: 0.9368 - loss: 0.1646 - val_accuracy: 0.9390 - val_loss: 0.1617
Epoch 7/10
1305/1305 ━━━━━━━━━━━━━━━━━━━━ 35s 27ms/step - accuracy: 0.9423 - loss: 0.1526 - val_accuracy: 0.9222 - val_loss: 0.1976
Epoch 8/10
1305/1305 ━━━━━━━━━━━━━━━━━━━━ 39s 25ms/step - accuracy: 0.9406 -


Accuracy: 0.9373302289483896

Classification Report:
              precision    recall  f1-score   support

 non-suicide       0.94      0.93      0.94     23191
     suicide       0.93      0.94      0.94     23195

    accuracy                           0.94     46386
   macro avg       0.94      0.94      0.94     46386
weighted avg       0.94      0.94      0.94     46386


ANN model and tokenizer saved successfully
